In [1]:
import config
from zipfile import ZipFile
from datatable import dt, f, by
from tqdm import tqdm
from collections import Counter
import pandas as pd

In [2]:
def get_path_db_zip(db):
	db_zip = f'{config.PATH_DATABASES}/{db}/{db}.zip'
	if db == 'SIHSUS':
		db_zip = '/Volumes/SanDisk/SIHSUS.zip'
	return db_zip

In [3]:
def get_files_from_db_zip(db_zip):
	return ZipFile(db_zip).namelist()

In [4]:
def get_cnes_list(db, df):
	cnes_list = list()
	if db == 'CNES':
		cnes_list = df[f.VINC_SUS == 1, 'CNES'].to_list()[0]
	if db == 'SINASC' and 'CODESTAB' in df.names:
		df['CODESTAB'] = dt.Type.str32
		cnes_list = df[
		(
			(f['ano_nasc'] >= 2010)
			&
			(f['ano_nasc'] <= 2019)
		) & (
			(f['IDADEMAE'] >= 10)
			&
			(f['IDADEMAE'] <= 49)
		) & (
			(f['res_RSAUDCOD'] != None)
			&
			(f['res_RSAUDCOD'] != 5301)
		) & (
			(f['nasc_RSAUDCOD'] != None)
		) & (
			(f['res_codigo_adotado'] != None)
		) & (
			(f['nasc_codigo_adotado'] != None)
		) & (
			(f['def_loc_nasc'] == 'Hospital')
		) & (
			(f['def_parto'] != 'Ignorado')
		) & (
			(f['CODESTAB'] != None)
		), 'CODESTAB']
		cnes_list = cnes_list[0, :, by('CODESTAB')].to_list()[0]
	if db == 'SIHSUS' and 'CNES' in df.names:
		cnes_list = df[
		(
			(f['PROC_REA'] == 310010039)
			|
			(f['PROC_REA'] == 411010034)
		) & (
			(f['ano_internacao'] >= 2010)
			&
			(f['ano_internacao'] <= 2019)
		), 'CNES']
		cnes_list = cnes_list[0, :, by('CNES')].to_list()[0]
	return cnes_list

In [5]:
def open_ith_file_in_db_zip(db, i):
	db_zip = get_path_db_zip(db)
	files = get_files_from_db_zip(db_zip)
	l = [False for _ in range(236)]
	l.append(True)
	return dt.fread(f'{db_zip}/{files[i]}', columns=l)

In [6]:
def get_all_cnes(db):
	zeros = '0000000'
	possible_columns = {
		'CNES', 'VINC_SUS', 'CODESTAB', 'PROC_REA',
		'ano_internacao', 'ano_nasc', 'IDADEMAE',
		'res_RSAUDCOD', 'nasc_RSAUDCOD',
		'res_codigo_adotado', 'nasc_codigo_adotado',
		'def_loc_nasc', 'def_parto'}
	db_zip = get_path_db_zip(db)
	files = get_files_from_db_zip(db_zip)
	s = set()
	for fname in tqdm(files):
		if 'dict' in fname: continue
		f_path = f'{db_zip}/{fname}'
		colums = dt.fread(f_path, max_nrows=0).names
		colums = [col in possible_columns for col in colums]
		df = dt.fread(f_path, columns=colums)
		cnes_list = get_cnes_list(db, df)
		s.update({(zeros+str(cnes))[-7:] for cnes in cnes_list if cnes})
	return s

In [7]:
def save_cnes_dbs(dbs):
	for db in dbs:
		df = get_all_cnes(db)
		pd.DataFrame(df, columns=['cnes']).sort_values(
			'cnes'
		).to_csv(
			f'data/consult/cnes/{db}_cnes.csv',
			index=False
		)

100%|██████████| 675/675 [01:49<00:00,  6.15it/s]

100%|██████████| 4375/4375 [03:54<00:00, 18.67it/s]

100%|██████████| 4401/4401 [17:08<00:00,  4.28it/s]

In [8]:
dbs = ['SINASC'] # ['SINASC', 'CNES', 'SIHSUS']
save_cnes_dbs(dbs)

100%|██████████| 675/675 [04:12<00:00,  2.67it/s]


In [9]:
cnes = set(pd.read_csv('data/consult/cnes/CNES_cnes.csv', dtype={'cnes':str})['cnes'])
sih = set(pd.read_csv('data/consult/cnes/SIHSUS_cnes.csv', dtype={'cnes':str})['cnes'])
sinasc = set(pd.read_csv('data/consult/cnes/SINASC_cnes.csv', dtype={'cnes':str})['cnes'])

In [24]:
pd.DataFrame(cnes.intersection(sinasc), columns=['cnes']).to_csv(
	'data/consult/cnes/cnes_sus_sinasc.csv',
	index=False
)

In [10]:
len(sih.intersection(sinasc))#/len(sih)

4093

In [11]:
len(sih.intersection(cnes))/len(sih)

0.999277108433735

In [12]:
len(sih)

4150

In [13]:
len(sih.intersection(cnes))

4147

- cnes no sinasc:
10370

- cnes que ta no sinasc e na base do cnes com vínculo com sus:
9084

- ou seja, 87% dos cnes que estão no sinasc possuem vínculo com sus

dos 4150 cnes que estão no sih, apenas 3 não estão na base do cnes com vínculo com sus

In [15]:
df = open_ith_file_in_db_zip('SINASC', 100)
df

ValueError: Input contains 85 columns, whereas columns parameter specifies only 237 columns